In [ ]:
"""
TO DO:

- add wind power production -> Andre, Joschi
- add distingtion betweent taks priority -> Tanver, Lorenz
- data reasearch: price per battery capacity (mx a+mx), solar panel, small vertical wind turbine -> Emanuele
- plots
- summer/winter
- (co2 equivalent)
- (monetization)
- (sensitivity analysis)
- (dynamic importance of tasks)
- add facebook battery model

"""

<h2>Imports</h2>

In [32]:
# Import libraries

import simpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from util import Battery, read_battery_process, solar_charge_process, compute_tasks_process, wind_charge_process

<h2>Constraints</h2>

In [ ]:
# Set constraints

minimal_reliability = 0.99

<h2>Data prep</h2>

In [2]:
# Load and prepare data from csv-files
START_DATE = pd.to_datetime("2021-06-01 00:00:00")
END_DATE = pd.to_datetime("2021-06-08 00:00:00")
INDEX = pd.date_range(START_DATE, END_DATE, freq="s")  # +timedelta(minutes=1)


# Load data between START_DATE and END_DATE
with open("data/weather_berlin_2021-06.csv", "r") as csvfile:
    weather_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/low_prio_tasks.csv", "r") as csvfile:
    tasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]
with open("data/high_prio_tasks.csv", "r") as csvfile:
    hptasks_df = pd.read_csv(csvfile, parse_dates=True, index_col=0)[START_DATE:END_DATE]

<h2>Scenario and costs</h2>

In [31]:
# Set researched values for costs etc. depending on the scenario

cost_per_capacity = 112*10**-3         # €/Wh
cost_per_solar = 330

BATTERY_CAPACITIES = list(np.linspace(500, 6000, 2))     # Wh

SOLAR_AREAS = [i*1.7 for i in range(4)]      # m^2
wind_areas = ["large", "small", "none"]
wind_prices = [400, 2000]                                # €

<h2>Simulation Function</h2>

In [35]:
def simulation(panel, capacity, area):
    """
    Runs simulation for a specific setup.
    """

    # Calculate set-up cost
    cost = cost_per_capacity*capacity + cost_per_solar*panel/1.7
    if area=="large":
        cost += wind_prices[1]
    elif area=="small":
        cost +=wind_prices[0]

    # Initialize Battery
    battery = Battery(capacity=capacity, soc=capacity)
    battery_socs = []
    task_stats = []

    # Run simulation
    env = simpy.Environment()
    env.process(solar_charge_process(env, battery, solar_area=panel, solar_values=weather_df["solar"]))
    env.process(wind_charge_process(env, battery, area, weather_df["wind"]))
    env.process(compute_tasks_process(env, battery, start_date=START_DATE, task_arrival_times=tasks_df.index, task_stats=task_stats))
    env.process(read_battery_process(env, battery, battery_socs))
    env.run(until=len(INDEX))

    # Calculate stats
    rate = check_stats(task_stats)
    print("____config done____")

    return cost, rate



def check_stats(stats):
    """
    Calculates successrate from stats.
    """

    success_count = 0

    for task in stats:
        if task[1]:
            success_count += 1

    success_rate = success_count/len(stats)

    return success_rate



In [6]:
def wind_charge_process(env, battery, wind_speed, wind_area, wind_efficiency=0.25):
    """Simpy process that charges the battery on every timestep."""
    for wind in wind_speed:
        for i in range(60):
            battery.update(0.5 * 1.225 * (wind_speed**3) * wind_area * wind_efficiency / 3600)
            yield env.timeout(1)

<h2>Main</h2>

In [33]:
#results = np.zeros((len(BATTERY_CAPACITIES), len(SOLAR_AREAS), 2))
columns = ['Panel count', 'Battery capacity', 'Windturbine size', 'Price', 'Success rate']
result_df = pd.DataFrame(columns=columns)


for area in wind_areas:    
    for count_p, panel in enumerate(SOLAR_AREAS):
        for count_b, capacity in enumerate(BATTERY_CAPACITIES):

            cost, rate = simulation(panel, capacity, area)

            df_entry = pd.DataFrame([[panel/1.7, capacity, area, cost, rate]], columns=columns)
            print(df_entry)
            
            result_df = pd.concat([result_df, df_entry])
            print(result_df)
            #results[count_b, count_p, 0] = cost
            #results[count_b, count_p, 1] = rate
            if count_b==len(BATTERY_CAPACITIES):
                print('finished column')

print(result_df)

test
wind area:  large
____config done____
   Panel count  Battery capacity Windturbine size   Price  Success rate
0          0.0             500.0            large  2056.0      0.076849
   Panel count  Battery capacity Windturbine size   Price  Success rate
0          0.0             500.0            large  2056.0      0.076849
test
wind area:  large
____config done____
   Panel count  Battery capacity Windturbine size   Price  Success rate
0          0.0            6000.0            large  2672.0      0.547212
   Panel count  Battery capacity Windturbine size   Price  Success rate
0          0.0             500.0            large  2056.0      0.076849
0          0.0            6000.0            large  2672.0      0.547212
test
wind area:  large
____config done____
   Panel count  Battery capacity Windturbine size   Price  Success rate
0          1.0             500.0            large  2386.0      0.811551
   Panel count  Battery capacity Windturbine size   Price  Success rate
0      

<h2>Results</h2>

In [34]:
result_df.head(100)

,Panel count,Battery capacity,Windturbine size,Price,Success rate
0,0.0,500.0,large,2056.0,0.076849
0,0.0,6000.0,large,2672.0,0.547212
0,1.0,500.0,large,2386.0,0.811551
0,1.0,6000.0,large,3002.0,1.000000
0,2.0,500.0,large,2716.0,0.936811
0,2.0,6000.0,large,3332.0,1.000000
0,3.0,500.0,large,3046.0,0.959099
0,3.0,6000.0,large,3662.0,1.000000
0,0.0,500.0,small,456.0,0.047532
0,0.0,6000.0,small,1072.0,0.496086
